In [1]:
import importlib
import torch

import utils.Constants

from dataLoader.DataLoader import get_data_loader
from utils.Constants import tokenizer

from models.QABert import QABertTrainer
from train.Trainer import train_epoch

In [2]:
if torch.cuda.is_available():
    device = torch.cuda.current_device()
    print('Current device:', torch.cuda.get_device_name(device))
else:
    print('Failed to find GPU. Will use CPU.')
    device = 'cpu'


Failed to find GPU. Will use CPU.


In [3]:
import dataLoader.DataLoader
importlib.reload(dataLoader.DataLoader)
from dataLoader.DataLoader import get_data_loader

dev_data_loader = get_data_loader(data_type = 'dev', tokenizer = tokenizer, batch_size = 1)

100%|██████████| 8421/8421 [00:37<00:00, 224.82it/s]


We have 8421 dev data with 2716 tables.


In [56]:
import models.AggregationClassifier

importlib.reload(models.AggregationClassifier)
importlib.reload(utils.Constants)

from models.AggregationClassifier import AggregationClassifierTrainer

agg_class_trainer = AggregationClassifierTrainer(device, dev_data_loader)

d = next(iter(dev_data_loader))

input_ids = d["input_ids"].to(device)
attention_mask = d["attention_mask"].to(device)
token_type_ids = d["token_type_ids"].to(device)

agg_target = d["target"]['SELECT_AGG'].to(device)

agg_output = agg_class_trainer.predict(
    input_ids,
    attention_mask,
    token_type_ids,
    agg_target.view(-1)
)

loss = agg_class_trainer.calc_loss(
    agg_output, agg_target
)

print(loss)

None


In [4]:
import models.SelectRanker
import models.WhereRanker
import train.Trainer
import models.AggregationClassifier
import models.WhereConditionClassifier

importlib.reload(models.QABert)
importlib.reload(models.SelectRanker)
importlib.reload(models.WhereRanker)
importlib.reload(models.WhereConditionClassifier)
importlib.reload(train.Trainer)

from models.QABert import QABertTrainer
from models.SelectRanker import SelectRankerTrainer
from models.WhereRanker import WhereRankerTrainer
from models.WhereConditionClassifier import WhereConditionClassifierTrainer
from models.AggregationClassifier import AggregationClassifierTrainer
from train.Trainer import train_epoch

models = dict(
    selection_trainer = SelectRankerTrainer(device, dev_data_loader),
    agg_class_trainer = AggregationClassifierTrainer(device, dev_data_loader),
    where_ranker_trainer = WhereRankerTrainer(device, dev_data_loader),
    where_cond_class_trainer = WhereConditionClassifierTrainer(device, dev_data_loader),
    qa_trainer = QABertTrainer(device, dev_data_loader),
)

train_epoch(models, dev_data_loader, device)

  0%|          | 0/8421 [00:13<?, ?batch/s]


RuntimeError: Trying to backward through the graph a second time, but the saved intermediate results have already been freed. Specify retain_graph=True when calling .backward() or autograd.grad() the first time.